# LangGraph: Tool Integration & Persistent Memory

This notebook demonstrates advanced LangGraph patterns:
1. **Tool Binding**: Connecting external tools (web search, notifications) to LangGraph agents
2. **Cyclic Graphs**: Implementing feedback loops where agents can call tools and re-process results
3. **Checkpointing**: Persisting conversation state across invocations using `MemorySaver` and `SqliteSaver`

**Key Concept:** LangGraph's "super-step" architecture requires explicit checkpointing to maintain state between invocations.

In [ ]:
# Import dependencies
from typing import Annotated
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langgraph.checkpoint.sqlite import SqliteSaver
import requests
import os
import sqlite3
from langchain_openai import ChatOpenAI
from langchain.agents import Tool
from langchain_community.utilities import GoogleSerperAPIWrapper
from typing import TypedDict

In [ ]:
# Initialize Environment
load_dotenv(override=True)

## Phase 1: Define Tools

Using LangChain's `Tool` wrapper to create agent-compatible functions.

In [ ]:
# Web Search Tool (via Serper API)
serper = GoogleSerperAPIWrapper()
tool_search = Tool(
    name="search",
    func=serper.run,
    description="Search the web for current information"
)

# Test
tool_search.invoke("What is the capital of France?")

In [ ]:
# Push Notification Tool
def push(text: str):
    """Send push notification via Pushover"""
    token = os.getenv("PUSHOVER_TOKEN")
    user = os.getenv("PUSHOVER_USER")
    if token and user:
        requests.post("https://api.pushover.net/1/messages.json", 
                      data={"token": token, "user": user, "message": text})

tool_push = Tool(
    name="send_push_notification",
    func=push,
    description="Send a push notification to the user"
)

tools = [tool_search, tool_push]

## Phase 2: Build Cyclic Graph with Tools

In [ ]:
# Define State
class State(TypedDict):
    messages: Annotated[list, add_messages]

# Initialize Graph
graph_builder = StateGraph(State)

# LLM with Tool Bindings
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

# Chatbot Node
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# Add Nodes
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))

# Add Edges (Cyclic: chatbot ↔ tools)
graph_builder.add_conditional_edges("chatbot", tools_condition, "tools")
graph_builder.add_edge("tools", "chatbot")  # Return to chatbot after tool execution
graph_builder.add_edge(START, "chatbot")

# Compile (No Memory Yet)
graph = graph_builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Test Execution (Stateless)
def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]})
    return result["messages"][-1].content

gr.ChatInterface(chat, type="messages").launch()

## Phase 3: Add Memory (In-Memory Checkpointing)

**Why Checkpointing?** LangGraph's state is scoped to a single invocation ("super-step"). To maintain context across multiple user interactions, we need persistence.

In [ ]:
# Configure Memory
memory = MemorySaver()

# Rebuild Graph with Checkpointing
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))
graph_builder.add_conditional_edges("chatbot", tools_condition, "tools")
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

graph = graph_builder.compile(checkpointer=memory)
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Execute with Memory (Thread-based Persistence)
config = {"configurable": {"thread_id": "1"}}

def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return result["messages"][-1].content

gr.ChatInterface(chat, type="messages").launch()

In [ ]:
# Inspect State
graph.get_state(config)

In [ ]:
# View State History (Time-travel Debugging)
list(graph.get_state_history(config))

## Phase 4: Persistent Memory (SQLite Checkpointing)

Upgrading to database-backed persistence for production resilience.

In [ ]:
# Configure SQLite Checkpointer
db_path = "memory.db"
conn = sqlite3.connect(db_path, check_same_thread=False)
sql_memory = SqliteSaver(conn)

# Rebuild Graph with SQL Persistence
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))
graph_builder.add_conditional_edges("chatbot", tools_condition, "tools")
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

graph = graph_builder.compile(checkpointer=sql_memory)
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Execute with Persistent Memory
config = {"configurable": {"thread_id": "3"}}

def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return result["messages"][-1].content

gr.ChatInterface(chat, type="messages").launch()